In [1]:
import sys
import os
import logging
from datetime import datetime, date
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
from sqlalchemy import create_engine
import json
from dotenv import load_dotenv

# プロジェクトのルートディレクトリをパスに追加
sys.path.append('..')

# ログ設定
logging.basicConfig(
    level=logging.WARNING, # WARNING以上のみ表示（INFO, DEBUGを非表示）
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# 環境変数読み込み
load_dotenv()

print("=== 競馬レースデータ収集システム ===")
print(f"実行時刻: {datetime.now()}")

# 必要なモジュールをインポート
from src.scraping.scrapers.race_scraper import RaceScraper
from src.scraping.scrapers.horse_scraper import HorseScraper
from src.scraping.scrapers.jockey_scraper import JockeyScraper
from src.scraping.scrapers.trainer_scraper import TrainerScraper
from src.database.schemas.race_schema import Race, RaceResult
from src.scraping.storage.postgresql_storage import PostgreSQLStorage

print("✅ システム初期化完了")



=== 競馬レースデータ収集システム ===
実行時刻: 2025-09-04 16:54:23.040330
✅ システム初期化完了


In [2]:
# =============================================================================
# 1. 接続設定
# =============================================================================

print("🔌 データベース接続を設定中...")

# PostgreSQL接続設定
pg_config = {
    'host': os.getenv('POSTGRES_HOST', 'localhost'),
    'database': os.getenv('POSTGRES_DB', 'stallion_db'),
    'user': os.getenv('POSTGRES_USER', 'stallion_user'),
    'password': os.getenv('POSTGRES_PASSWORD'),
    'port': int(os.getenv('POSTGRES_PORT', '5432'))
}

# SQLAlchemy エンジン作成（pandasで使用）
DATABASE_URL = f"postgresql://{pg_config['user']}:{pg_config['password']}@{pg_config['host']}:{pg_config['port']}/{pg_config['database']}"
engine = create_engine(DATABASE_URL)

print("✅ 接続設定完了")


# pandas
def query(sql):
    """SQLクエリを実行してDataFrameで結果を返す"""
    return pd.read_sql(sql, engine)


🔌 データベース接続を設定中...
✅ 接続設定完了


In [5]:
# スクレイパーとストレージを初期化
scraper = JockeyScraper(delay=2.0)  # 2秒間隔でリクエスト
storage = PostgreSQLStorage()

jockeys = scraper.scrape_jockeys(limit=1000)
print(f"取得した騎手数: {len(jockeys)}")
print(jockeys)

ページ 1 を取得中... (残り 1000 件)
このページで 100 名の騎手を取得しました
ページ 2 を取得中... (残り 900 件)
このページで 100 名の騎手を取得しました
ページ 3 を取得中... (残り 800 件)
このページで 100 名の騎手を取得しました
ページ 4 を取得中... (残り 700 件)
このページで 100 名の騎手を取得しました
ページ 5 を取得中... (残り 600 件)
このページで 100 名の騎手を取得しました
ページ 6 を取得中... (残り 500 件)
このページで 100 名の騎手を取得しました
ページ 7 を取得中... (残り 400 件)
このページで 54 名の騎手を取得しました
合計 654 名の騎手データを取得しました
取得した騎手数: 654
[Jockey(jockey_id='05386', name_ja='戸崎圭太', name_en=None, birthdate=datetime.date(1980, 7, 8), region='東', license_type=None, trainer_name='田島俊明', debut_date=None, status='active', weight=None, height=None, total_races=531, wins=90, seconds=73, thirds=56, win_rate=Decimal('16.95'), show_rate=Decimal('41.24'), total_prize_money=Decimal('1783303000.0'), yearly_stats={'2025': {'races': 531, 'wins': 90, 'seconds': 73, 'thirds': 56, 'representative_horse': 'ダノンデサイル'}}, track_stats={'turf': {'entries': 303, 'wins': 50}, 'dirt': {'entries': 228, 'wins': 40}}, distance_stats=None, created_at=datetime.datetime(2025, 9, 4, 16, 56, 5

In [6]:
for jockey in jockeys:
    storage.insert_jockey(jockey)
    print (f"Inserted jockey: {jockey.name_ja} ({jockey.jockey_id})")

Inserted jockey: 戸崎圭太 (05386)
Inserted jockey: 松山弘平 (01126)
Inserted jockey: 坂井瑠星 (01163)
Inserted jockey: 横山武史 (01170)
Inserted jockey: Ｃ．ルメール (05339)
Inserted jockey: 川田将雅 (01088)
Inserted jockey: 丹内祐次 (01091)
Inserted jockey: 佐々木大輔 (01197)
Inserted jockey: 岩田望来 (01174)
Inserted jockey: 横山和生 (01140)
Inserted jockey: 武豊 (00666)
Inserted jockey: 北村友一 (01102)
Inserted jockey: 団野大成 (01180)
Inserted jockey: 高杉吏麒 (01213)
Inserted jockey: 鮫島克駿 (01157)
Inserted jockey: 藤岡佑介 (01093)
Inserted jockey: 幸英明 (00732)
Inserted jockey: 吉村誠之助 (01216)
Inserted jockey: 三浦皇成 (01122)
Inserted jockey: 菅原明良 (01179)
Inserted jockey: 津村明秀 (01092)
Inserted jockey: 荻野極 (01160)
Inserted jockey: 大野拓弥 (01096)
Inserted jockey: 横山典弘 (00660)
Inserted jockey: 小林美駒 (01206)
Inserted jockey: 岩田康誠 (05203)
Inserted jockey: 池添謙一 (01032)
Inserted jockey: 田口貫太 (01208)
Inserted jockey: 小沢大仁 (01185)
Inserted jockey: 斎藤新 (01178)
Inserted jockey: 浜中俊 (01115)
Inserted jockey: 西村淳也 (01171)
Inserted jockey: 田辺裕信 (01075)
Inserted joc

---
以下、調教師